# Initialization

In [25]:
%reload_ext autoreload
%autoreload 2

In [26]:
from result_saver import SaverProvider

provider = SaverProvider()

# Initialize simulator

In [57]:
from soft_info import RepCodeIQSimulator

DEVICE = 'ibm_sherbrooke'
DISTANCE = 30
ROUNDS = 30
OTHER_DATE = '2023-10-27'
_RESETS = False
LOGICAL = 0 # NOT NEEDED FOR EXTREME IQ BCS HARDCODED 0

_is_hex = True
if DEVICE == 'ibmq_mumbai':
    _is_hex = False

# Initialize simulator
simulator = RepCodeIQSimulator(provider, DISTANCE, ROUNDS, DEVICE, _is_hex=_is_hex, _resets = _RESETS, other_date=OTHER_DATE)

Found jobs for backend ibm_sherbrooke with closest execution date 2023-10-27 08:32:22.841567+00:00.
Found jobs for backend ibm_sherbrooke with closest execution date 2023-10-27 08:32:22.841567+00:00.
Searching for ibm_sherbrooke and 23.10.27_07h46_300pts_2std


# Dev get IQ data

In [59]:
import numpy as np

SHOTS = int(1e3)
NOISE_LIST = [3e-2, 0.8e-2, 1e-2, 3e-2] # [two-qubit-fidelity, reset error, measurement error, idle error]


stim_circuit = simulator.get_stim_circuit(NOISE_LIST)
meas_outcomes = stim_circuit.compile_sampler(seed=42).sample(SHOTS)
print("generated counts")

if not simulator._resets:
    # LOGIC
    pass

IQ_memory = np.zeros_like(meas_outcomes, dtype=np.complex128)
len_IQ_array = len(simulator.qubit_mapping)
kde_samples_needed = {qubit_idx: {'0': 0, '1': 0} for qubit_idx in simulator.kde_dict.keys()}
sample_counters = {qubit_idx: {'0': 0, '1': 0} for qubit_idx in simulator.kde_dict.keys()}    

print("starting kde samples needed")
for row in meas_outcomes:
    for IQ_idx, bit in enumerate(row):
        qubit_idx = simulator.qubit_mapping[IQ_idx]
        kde_samples_needed[qubit_idx][str(int(bit))] += 1

print("starting kde samples")
kde_samples = {}
for qubit_idx, needed_nb_samples in kde_samples_needed.items():
    [kde0, kde1], scaler = simulator.kde_dict[qubit_idx], simulator.scaler_dict[qubit_idx]
    if needed_nb_samples['0'] > 0:
        samples0 = scaler.inverse_transform(kde0.sample(needed_nb_samples['0'], random_state=42))
    else:
        samples0 = np.empty((0, 2)) 
    if needed_nb_samples['1'] > 0:
        samples1 = scaler.inverse_transform(kde1.sample(needed_nb_samples['1'], random_state=42))
    else:
        samples1 = np.empty((0, 2)) 
    kde_samples[qubit_idx] = {'0': samples0, '1': samples1}

print("starting IQ memory")
for row_idx, row in enumerate(meas_outcomes):
    for IQ_idx, bit in enumerate(row):
        qubit_idx = simulator.qubit_mapping[IQ_idx]
        sample_index = sample_counters[qubit_idx][str(int(bit))]
        sample = kde_samples[qubit_idx][str(int(bit))][sample_index]   
        IQ_memory[row_idx, IQ_idx] = complex(sample[0], sample[1])
        sample_counters[qubit_idx][str(int(bit))] += 1

assert sample_counters == kde_samples_needed

print(IQ_memory.shape)
IQ_memory

generated counts
starting kde samples needed
starting kde samples
starting IQ memory
(1000, 900)


array([[-5.18768295e+06+2.63782954e+07j,  4.03449489e+07+8.21901591e+07j,
         4.11237248e+07-7.23546076e+07j, ...,
        -7.61349458e+06-8.86157230e+06j, -1.00103038e+07-6.34246613e+06j,
        -1.18927224e+07-6.54485218e+06j],
       [-1.50394372e+07+1.64640372e+07j, -9.44620903e+07+6.48695364e+07j,
        -5.07620938e+07-7.43007491e+07j, ...,
        -5.99718020e+06-6.75866903e+06j,  5.54395063e+06-9.09808474e+06j,
         2.42216856e+06-5.71829960e+06j],
       [-7.63041532e+06+2.66804186e+07j, -7.25839250e+07+5.78922010e+07j,
        -7.28451288e+07-9.28146204e+07j, ...,
        -7.63357051e+06-8.55444300e+06j, -3.94810537e+06-9.28868674e+06j,
        -6.54017650e+06-8.06684671e+06j],
       ...,
       [ 1.71499938e+07+1.46959760e+07j, -5.72242184e+07+8.44821241e+07j,
        -4.61100562e+07-1.02343032e+08j, ...,
        -5.55623884e+06-6.43533041e+06j,  7.19104994e+06-1.07144996e+07j,
        -3.90299078e+06-6.44391675e+06j],
       [-8.38007770e+06+2.05636444e+07j, -4.

In [72]:
import numpy as np

SHOTS = int(1e5)
NOISE_LIST = [3e-2, 0.8e-2, 1e-2, 3e-2] # [two-qubit-fidelity, reset error, measurement error, idle error]


stim_circuit = simulator.get_stim_circuit(NOISE_LIST)
meas_outcomes = stim_circuit.compile_sampler(seed=42).sample(SHOTS)
print("generated counts")

if not simulator._resets:
    # LOGIC
    pass

IQ_memory = np.zeros_like(meas_outcomes, dtype=np.complex128)
# print(f"IQ_memory.shape: {IQ_memory.shape}")
# CORRECT 


len_IQ_array = len(simulator.qubit_mapping)
# print(f"len_IQ_array: {len_IQ_array}")
# CORRECT

used_qubits = set(simulator.qubit_mapping.values())
kde_samples_needed = {qubit_idx: {'0': 0, '1': 0} for qubit_idx in simulator.kde_dict.keys()}
sample_counters = {qubit_idx: {'0': 0, '1': 0} for qubit_idx in simulator.kde_dict.keys()}    
# print("qubit_mapping: ", simulator.qubit_mapping)  
# CORRECT



print("starting kde samples needed")
for IQ_idx in range(len_IQ_array):
    qubit_idx = simulator.qubit_mapping[IQ_idx]  # Find the corresponding qubit index
    # Get the binary outcomes for this qubit across all shots
    outcomes = meas_outcomes[:, IQ_idx]
    # Use np.bincount to count '0's and '1's. Ensure minlength=2 for the case all outcomes are the same.
    counts = np.bincount(outcomes, minlength=2)
    # print(f"counts: {counts} at IQ_idx: {IQ_idx} and qubit_idx: {qubit_idx}")
    # Now, update kde_samples_needed directly with these counts.
    kde_samples_needed[qubit_idx]['0'] += counts[0]
    kde_samples_needed[qubit_idx]['1'] += counts[1]

# print(f"kde_samples_needed: {kde_samples_needed}")

# # add up the samples needed for each qubit
# total = 0
# for k,v in kde_samples_needed.items():
#      total += v['0'] + v['1']

# print(f"total: {total}")



print("starting kde samples")
kde_samples = {}
for qubit_idx, needed_nb_samples in kde_samples_needed.items():
    [kde0, kde1], scaler = simulator.kde_dict[qubit_idx], simulator.scaler_dict[qubit_idx]
    if needed_nb_samples['0'] > 0:
        samples0 = scaler.inverse_transform(kde0.sample(needed_nb_samples['0'], random_state=42))
    else:
        samples0 = np.empty((0, 2)) 
    if needed_nb_samples['1'] > 0:
        samples1 = scaler.inverse_transform(kde1.sample(needed_nb_samples['1'], random_state=42))
    else:
        samples1 = np.empty((0, 2)) 
    kde_samples[qubit_idx] = {'0': samples0, '1': samples1}

print("starting IQ memory")
next_sample_indices = np.zeros((len(simulator.kde_dict), 2), dtype=int)

# Iterate over meas_outcomes to fill IQ_memory
for row_idx, row in enumerate(meas_outcomes):
    for IQ_idx, bit in enumerate(row):
        qubit_idx = simulator.qubit_mapping[IQ_idx]
        bit_int = int(bit)
        
        # Get the next sample index for this qubit and bit
        sample_index = next_sample_indices[qubit_idx, bit_int]
        
        # Retrieve the sample and update IQ_memory
        sample = kde_samples[qubit_idx][str(bit_int)][sample_index]
        IQ_memory[row_idx, IQ_idx] = complex(sample[0], sample[1])
        
        # Increment the sample index for the next use
        next_sample_indices[qubit_idx, bit_int] += 1

print(IQ_memory.shape)
IQ_memory

generated counts
starting kde samples needed
starting kde samples
starting IQ memory
(100000, 900)


array([[-17355641.98719694+2.63384060e+07j,
        -97294791.54091156+6.09645592e+07j,
        -99708423.8111868 -8.48527765e+07j, ...,
        -10374803.17465698-9.09585021e+06j,
          5223542.76157181-8.69859212e+06j,
          3159480.9014161 -5.30116596e+06j],
       [-19120629.3411788 +2.50096776e+07j,
        -37353578.80362493+3.94946990e+07j,
        -72151782.15198714-8.16098874e+07j, ...,
         -4473883.9810702 -6.60640595e+06j,
         -8405949.08248057-7.49053715e+06j,
          8936334.39095261-6.19866188e+06j],
       [-21051486.39020458+2.63787359e+07j,
         60386235.75964344+6.61283530e+07j,
        -79171146.76510184-1.10424387e+08j, ...,
         -5365550.39691601-8.45003923e+06j,
          3277069.84994949-3.08243709e+06j,
        -12719133.44982099-7.53988566e+06j],
       ...,
       [-12553569.47431455+1.97122555e+07j,
        -40321600.25995508+6.74130439e+07j,
        -78996219.16429117-1.19549844e+08j, ...,
          7090371.9068215 -8.18226907e+06

In [73]:
IQ_data = simulator.generate_IQ(SHOTS, NOISE_LIST)
print(IQ_data.shape)
# IQ_data

generated stim counts
correcting counts
finished correcting counts
(100000, 900)


# Get IQ data

In [63]:
IQ_data = simulator.generate_IQ(SHOTS, noise_list=NOISE_LIST)

generated stim counts
correcting counts
finished correcting counts
